In [3]:
import findspark

findspark.init('/opt/spark')

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('lec_23_moynihan_ex1').getOrCreate()

22/04/22 11:32:28 WARN Utils: Your hostname, GPUServer resolves to a loopback address: 127.0.1.1; using 10.4.10.8 instead (on interface enp2s0)
22/04/22 11:32:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/04/22 11:32:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/22 11:32:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/04/22 11:32:29 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/04/22 11:32:29 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/04/22 11:32:29 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/04/22 11:32:29 W

In [6]:
# read text from your netcat socket

In [8]:
lines = spark.readStream\
.format('socket').option('host','localhost')\
.option('port',6969).load()

22/04/22 11:35:43 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [10]:
lines.printSchema()

root
 |-- value: string (nullable = true)



In [11]:
from pyspark.sql.functions import explode, split

In [16]:
words = lines.select(explode(split(lines.value, " ")).alias('words'))

In [18]:
#word count query

In [19]:
wordCounts = words.groupBy('words').count()

In [22]:
#start tbhed query

query = (wordCounts.writeStream
         .outputMode('complete')
         .format('memory')
         .queryName('wordCounts')
         .start())

22/04/22 11:45:38 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-5fbb306a-1846-4e45-8f7e-d1fd00ff4761. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [24]:
import time

wordCounts.createOrReplaceTempView('wc')

for x in range(10):
    df = spark.sql('SELECT * FROM wordCounts')
    df.show(10)
    time.sleep(5)

+-----+-----+
|words|count|
+-----+-----+
|  not|    1|
|  fun|    1|
| hate|    1|
|   is|    1|
|   it|    1|
|spark|    1|
|    i|    1|
|   at|    1|
|  all|    1|
|     |    1|
+-----+-----+

+-----+-----+
|words|count|
+-----+-----+
|  not|    1|
|  fun|    1|
| hate|    1|
|   is|    1|
|   it|    1|
|spark|    1|
|    i|    1|
|   at|    1|
|  all|    1|
|     |    1|
+-----+-----+

+-----+-----+
|words|count|
+-----+-----+
|  not|    1|
|  fun|    1|
| hate|    1|
|   is|    1|
|   it|    1|
|spark|    1|
|    i|    1|
|   at|    1|
|  all|    1|
|     |    1|
+-----+-----+

+-----+-----+
|words|count|
+-----+-----+
|  not|    1|
|  fun|    1|
| hate|    1|
|   is|    1|
|   it|    1|
|spark|    1|
|    i|    1|
|   at|    1|
|  all|    1|
|     |    1|
+-----+-----+

+-----+-----+
|words|count|
+-----+-----+
|  not|    1|
|  fun|    1|
| hate|    1|
|   is|    1|
|   it|    1|
|spark|    1|
|    i|    1|
|   at|    1|
|  all|    1|
|     |    1|
+-----+-----+

+-----+-----+
|

+-----+-----+
|words|count|
+-----+-----+
|  not|    1|
|  fun|    1|
| hate|    1|
|   is|    1|
|   it|    1|
|spark|    1|
|    i|    1|
|   at|    1|
|  all|    1|
|     |    1|
+-----+-----+

+-------+-----+
|  words|count|
+-------+-----+
|   even|    2|
|    not|    3|
|anymore|    1|
|    fun|    1|
|   hate|    1|
|     is|    4|
|    bad|    1|
|    any|    1|
|     it|    3|
|  spark|    2|
+-------+-----+
only showing top 10 rows



In [25]:
spark.streams.active[0].isActive

True

In [26]:
query.status

{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}

In [27]:
query.stop()

In [28]:
wordCounts.isStreaming

True